# First Attempt at Match Prediction

First attempt at predicting match outcomes for European Soccer data

The focus for simplicity will be on the EPL with the main aim to create the functions required to engineer the features

## 1. Imports



In [3]:
# mixture of libs for web scraping, parsing and pandas
import datetime as dt
import numpy as np
import pandas as pd
import seaborn as sns
import sqlite3
import warnings

pd.options.display.max_columns = None
warnings.filterwarnings('ignore')